# Go-Free Concept {-}

Simple GNSS clock bias model

In [1]:
from numpy import array, eye, printoptions
from numpy.linalg import inv

# Design matrix
H = array([[-0.7460,  0.4689, -0.4728, 1],
           [ 0.8607,  0.3446, -0.3747, 1],
           [-0.2109, -0.3503, -0.9126, 1],
           [ 0.0619,  0.4967, -0.8657, 1],
           [ 0.7249, -0.4760, -0.4980, 1],
           [ 0.4009, -0.1274, -0.9072, 1]])

# Difference matrix
B = array([[1, -1, 0, 0, 0, 0],
           [1, 0, -1, 0, 0, 0],
           [1, 0, 0, -1, 0, 0],
           [1, 0, 0, 0, -1, 0],
           [1, 0, 0, 0, 0, -1]])

# Measurement covariance matrix
R = 5**2*eye(6)

## State elimination {-}

In [2]:
# Design matrix (modified)
Hm = B@H
Hm = Hm[:,0:3]

# Measurement covariance matrix
Rm = B@R@B.T

# Inital state covariance matrix
P0 = array([[100**2, 0, 0],
            [0, 100**2, 0],
            [0, 0, 100**2]])

# Kalman gain
K = P0@Hm.T@inv(Hm@P0@Hm.T + Rm)

# Measurement update
P = (eye(3)-K@Hm)@P0@(eye(3)-K@Hm).T + K@Rm@K.T

with printoptions(precision=4, suppress=True):
    print(P)

[[ 17.0411   9.4092 -13.6752]
 [  9.4092  33.699  -19.5357]
 [-13.6752 -19.5357  96.252 ]]


## "Boosted" Q approach {-}

In [3]:
# Inital state covariance matrix
P0 = array([[100**2, 0, 0, 0],
            [0, 100**2, 0 ,0],
            [0, 0, 100**2, 0],
            [0, 0, 0, 10e12]])

# Kalman gain
K = P0@H.T@inv(H@P0@H.T + R)

# Measurement update
P = (eye(4)-K@H)@P0@(eye(4)-K@H).T + K@R@K.T

with printoptions(precision=4, suppress=True):
    print(P)

[[   17.0411     9.4092   -13.6752   -12.8465]
 [    9.4092    33.699    -19.5357   -16.8387]
 [  -13.6752   -19.5357    96.252     68.3138]
 [  -12.8465   -16.8387    68.3138 23448.6711]]
